<a href="https://colab.research.google.com/github/KiranH1007/-MLB-fan-engagement-project/blob/load-data-using-pandas/Live%20Game%20Sentiment%20Analysis/Live%20Game%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Authenticate Google Cloud in Colab
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Install Required Libraries
!pip install google-cloud-storage google-cloud-bigquery nltk pandas

In [83]:
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025' # Make sure this is the correct bucket name
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly without getting the bucket metadata first
# This assumes the bucket is public and blob_name includes the full path
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file
raw_data = blob.download_as_text()

# Print the data (optional)
#print(raw_data[0:100])

In [ ]:
# load json with pandas
# Data set is 2025-mlb-fan-favs-follows.json
import pandas as pd
import json
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file as bytes
data = blob.download_as_bytes()

# Decode the bytes to a string
raw_data = data.decode('utf-8')

# Load the data using json.loads to handle possible JSON Lines format
try:
    # Attempt to load as a single JSON object
    json_data = json.loads(raw_data)
    df = pd.DataFrame([json_data]) # Enclose in a list if it's a single object
except json.JSONDecodeError:
    # If single object fails, try loading as JSON Lines
    json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
    df = pd.DataFrame(json_data)

# Preview data
print(df.head())

print(df.info())

In [ ]:
# load json with pandas
# data set is datasets_mlb-fan-content-interaction-data_2025-mlb-fan-favs-follows-000000000000.json
import pandas as pd
import json
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
blob_name = 'datasets/mlb-fan-content-interaction-data/datasets_mlb-fan-content-interaction-data_2025-mlb-fan-favs-follows-000000000000.json'

# Get the blob directly
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file as bytes
data = blob.download_as_bytes()

# Decode the bytes to a string
raw_data = data.decode('utf-8')

# Load the data using json.loads to handle possible JSON Lines format
try:
    # Attempt to load as a single JSON object
    json_data = json.loads(raw_data)
    df2 = pd.DataFrame([json_data]) # Enclose in a list if it's a single object
except json.JSONDecodeError:
    # If single object fails, try loading as JSON Lines
    json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
    df2 = pd.DataFrame(json_data)

# Preview data
print(df2.head())

print(df2.info())


In [ ]:
# load json with pandas
# data set is mlb-fan-content-interaction-data-000000000000.json
import pandas as pd
import json
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
blob_name = 'datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-000000000000.json'

# Get the blob directly
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file as bytes
data = blob.download_as_bytes()

# Decode the bytes to a string
raw_data = data.decode('utf-8')

# Load the data using json.loads to handle possible JSON Lines format
try:
    # Attempt to load as a single JSON object
    json_data = json.loads(raw_data)
    df3 = pd.DataFrame([json_data]) # Enclose in a list if it's a single object
except json.JSONDecodeError:
    # If single object fails, try loading as JSON Lines
    json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
    df3 = pd.DataFrame(json_data)

# make single column for all content to content id due to similar data like slug and content_headline
'''
Redundancy: Storing essentially the same information twice.
Increased storage: Larger data size due to duplicated information.
Potential inconsistency: If one field is updated and the other isn't, it could create discrepancies.
'''
#step 1 observation
# print(len(df3['slug']))
# print(len(df3['content_headline']))
# print(df3['slug'][100])
# print(df3['content_headline'][100])
# print(df3['content_type'][100])

# step 2 combining
'''
Prioritize One Field:
If slug and content_headline convey nearly identical information, choose the one that's more concise and
consistently formatted (likely slug) as the primary identifier.
In most cases, the slug is a better choice for the primary identifier because:
Conciseness: Slugs are typically shorter and more URL-friendly than content headlines.
Consistency: Slugs are often generated programmatically and have a more consistent format compared to headlines, which might contain variations in punctuation and capitalization.
'''
# remove extra column
df3.drop(columns=['content_headline'], inplace=True)
df3['content_id'] = df3['content_type'] + ':' + df3['slug'].str.lower()
df3.drop(columns=['slug'], inplace=True)
df3.drop(columns=['content_type'], inplace=True)

#print(df3['content_id'][100])


# similarly for date time utc as date_time_date is same as utc
df3.drop(columns=['date_time_date'], inplace=True) # Drop the redundant column

# Preview data
df3.head()

#print(df3.info())

In [ ]:
# Analysis of DataFrames are named df, df2, df3
common_columns = list(set(df.columns) & set(df2.columns) & set(df3.columns))
print("Common Columns:", common_columns)


In [ ]:
#without cleanup and duplicate columns

import pandas as pd

# it should be outer for all unique data and inner for common data

# 1. Merge the first two datasets
merged_df = pd.merge(df, df2, on=common_columns, how='outer')

# 2. Merge the result with the third dataset which has content type headline
merged_df = pd.merge(merged_df, df3, on=common_columns, how='outer')

merged_df.count()

merged_df.head()

In [ ]:
# with cleanup and removal of duplicate columns

import pandas as pd

def merge_and_handle_duplicates_concise(df1, df2, on_columns):
    merged_df = pd.merge(df1, df2, on=on_columns, how='outer')
    for col in [c for c in merged_df if any(c.endswith(s) for s in ('_x', '_y'))]:
        other_col = col[:-2] + ('_y' if col.endswith('_x') else '_x')
        if other_col in merged_df:
            # Check if values are the same and have the same data type
            if merged_df[col].equals(merged_df[other_col]):

                # If same, keep one and drop the other
                merged_df = merged_df.drop(columns=[other_col])
                merged_df = merged_df.rename(columns={col: col[:-2]})  # Rename to remove suffix
            else:
                # If different (list type or other differences), apply previous logic
                if merged_df[col].apply(isinstance, args=(list,)).any() and merged_df[other_col].apply(isinstance, args=(list,)).any():
                    for c in (col, other_col):
                        merged_df[c] = merged_df[c].apply(lambda x: sorted(x) if isinstance(x, list) else x)
                merged_df[col[:-2]] = merged_df[col]  # Or apply a custom logic here
                merged_df = merged_df.drop(columns=[other_col])
    return merged_df

merged_df = merge_and_handle_jumbled_lists(df, df2, common_columns)

# removed manually as it has same values in favorite_team_id_x
merged_df = merged_df.drop(columns=['favorite_team_id_y'])

merged_df = pd.merge(merged_df, df3, on=common_columns, how='outer')


# BEFORE CLEANUPS stats
print(merged_df.isnull().sum())

# Remove rows with missing values in user_id
#merged_df.dropna(subset=['user_id'], inplace=True)

# Remove duplicates based on user_id keep only unique users
#merged_df.drop_duplicates(subset=['user_id'], inplace=True)
#print(merged_df.isnull().sum())

# Preview data
merged_df.head()

# defines column names and datatypes
#merged_df.info()